<a id='顶部'></a>
# 2018-11-09 写作判分项目
<br>
<br>
前期准备：[1.函数包。](#所需函数包)[2.全局变量](#所需全局变量)<br>
数据预处理：[1.读取数据。](#读取数据)[2.数据预处理。](#数据预处理)<br>
特征工程：[1.统计特征。](#统计特征)[2.语义特征。](#语义特征)[3.主题特征。](#主题特征)<br>
模型选择：[XGBoost](#XGBoost)<br>
分类不平衡：[Oversample](#过采样)<br>
评价函数：[F1score](#评价函数)<br>

<a id='所需函数包'></a>
### 相关函数包
[返回顶部](#顶部)

In [8]:
import os
import re
import nltk
import math
import gensim
import logging
import enchant
import nltk.data
import numpy as np
import pandas as pd
import xgboost as xgb
import multiprocessing
from sklearn import metrics
from collections import Counter
from nltk.corpus import stopwords
from gensim.models import LdaModel
from gensim.models import Word2Vec
from gensim import corpora, models 
from gensim.corpora import WikiCorpus
from nltk.tokenize import WordPunctTokenizer
from gensim.models.word2vec import LineSentence
from sklearn.cross_validation import train_test_split
logging.basicConfig(level=logging.DEBUG,  format='%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s') 

<a id='全局变量'></a>
### 声明全局变量
[返回顶部](#顶部)

In [9]:
global CHECK, STOP, STEM, IDF
CHECK = enchant.Dict("en_US")
STOP = stopwords.words('english')
STEM = nltk.stem.SnowballStemmer('english')

### 英文Wiki训练语料

In [10]:
# 英文WIKI数据读取
def Transfor():
    output_file="enwiki-latest-pages-articles.txt"
    if os.path.exists(output_file):
        output = open(output_file).readlines()
    else:
        inp="enwiki-latest-pages-articles.xml.bz2"
        i = 0
        output = open(output_file, 'w',encoding="utf-8")
        wiki = WikiCorpus(inp, lemmatize=False, dictionary={})
        for text in wiki.get_texts():
            output.write("".join(text) + "\n")
            i = i + 1
            if (i % 10000 == 0):
                print("Save "+str(i) + " articles")
        output.close()
    return output

<a id='文章分句'></a>
### 文章分句，保留每句话的长度
[上一级](#数据预处理)

In [11]:
# 分句，并保留每条句子长度
def SplitSentence(paragraph):
    paragraph = paragraph.replace("，",",")
    paragraph = paragraph.replace("。",'.')
    paragraph = paragraph.replace("？",'?')
    paragraph = paragraph.replace("！",'!')
    sentences = re.split(r'[,.?!]', paragraph)
    s = []
    i = 0
    for sentence in sentences:
        if sentence.strip() == '':
            continue
        c = len(sentence.split(' ')) + i
        i = 0
        if c == 1:
            i = 1
        s.append(c) 
    return s

<a id='粗分词'></a>
### 粗分词
[上一级](#统计特征)

In [12]:
# 分词，包括非英文
def Wordtokenizer(sentence):
    words = WordPunctTokenizer().tokenize(sentence)
    return words

<a id='细分词'></a>
### 细分词
[上一级](#统计特征)

In [13]:
# 只对英文进行分词
def Pure_word(line):
    return re.findall(r'[a-zA-Z]+', line)

<a id='去停用词'></a>
### 去停用词
[上一级](#数据预处理)

In [14]:
def Delete_stop(line):
    return [i.lower() for i in line if len(i.lower())>1 and i.lower() not in STOP and CHECK.check(i.lower())]

<a id='tfidf'></a>
### TFIDF保留关键词
[上一级](#数据预处理)

In [15]:
# 构建idf
def Idf(df):
    words = []
    for line in df['reserved_word']:
        words += list(set(line))
    return Counter(words)

# 求tfidf值 
def Tfidf(IDF, line):
    line = Counter(line)
    for key in line.keys():
        line[key] *= math.log(10561/(IDF[key] + 1))
    line = line.most_common(int(len(line)*0.5))
    line = [i[0] for i in line]
    return line

<a id='筛选词性'></a> 
### 筛选词性
[上一级](#数据预处理)

In [16]:
# 保留名词以及动词
def Pos_select(words):
    poswords = nltk.pos_tag(words)
    return [w[0] for w in poswords if 'N' in w[1] or 'V' in w[1] or 'JJ' in w[1]]

<a id='句子长度统计'></a>
### 句子长度三种统计量
[上一级](#统计特征)

In [17]:
# 统计句子长度
def Statics(line, method):
    if line == []:
        return 0
    elif method == 'max':
        return max(line)
    elif method == 'min':
        return min(line)
    elif method == 'mean':
        return np.mean(line)
    else:
        return 0

<a id='单词拼写'></a>
### 单词拼写
[上一级](#统计特征)

In [18]:
# 单词拼写情况
def Wrong_count(line):
    c = 0
    for word in line:
        if not CHECK.check(word):
            c += 1
    return c

<a id='错误标点'></a>
### 错误标点
[上一级](#统计特征)

In [19]:
# 统计错误标点使用
def Wrong_signal(line):
    return len(re.findall(r'[，。!！~·@#￥%……&*（）——+|《》？`$()_\、“”："><]', line))

<a id='首字母大写'></a>
### 首字母大写情况
[上一级](#统计特征)

In [20]:
# 判断句中首字母是否大写，“.？！”后接大写，“，”后字母接小写。 
def Initial_capitalization(text):
    c = 0
    text = text.replace(' ','')
    try:
        if text[0].islower():
            c += 1
        for i in range(len(text)-1):
            if text[i] in  ".!?" and text[i+1].islower():
                c += 1
            if text[i] in  "," and not text[i+1].islower():
                c += 1 
    except:
        pass
    return c

<a id='作文是否写完'></a>
### 作文是否写完
[上一级](#统计特征)

In [21]:
# 判断写作是否完成以“.”结尾表示写完，记作2；以其他符号结尾表示未写完，记作0；以名词结尾，忘记写句号，记作1；其他情况记作-1。
def End0(line):
    try:
        if line.endswith('.'):
            return 2
        elif 'N' in nltk.pos_tag(re.findall(r'[a-zA-Z]+', line)[-1]):
            return 1
        else:
            return 0
    except:
        return -1

<a id='字数限制'></a>
### 字数限制
[上一级](#统计特征)

In [22]:
# 各题目数字限制要求
def Limit_count(SysMeasureID, word):
    if SysMeasureID == 'C4M01':
        return word-120
    elif SysMeasureID == 'C4M02':
        return word-120
    elif SysMeasureID == 'C6M01':
        return word-80
    elif SysMeasureID == 'C6M02':
        return word-80
    elif SysMeasureID == 'C6M03':
        return word-80
    elif SysMeasureID == 'C4M03' and word < 120:
        return word-120
    elif SysMeasureID == 'C4M03' and word > 180:
        return word-180
    else:
        return word-120

<a id='用词丰富度'></a>
### 用词丰富度
[上一级](#统计特征)

In [23]:
def Richness(line):
    return len(set(line))/(len(line)+0.01)

<a id='词性统计'></a>
### 词性统计
[上一级](#统计特征)

In [24]:
def Tag_count(line):
    tag = {}; n = len(line)
    for i in line:
        t = nltk.pos_tag(i)[0][1]
        if t not in tag:
            tag[t] = 1/n
        else:
            tag[t] += 1/n
    return tag

<a id='word2vec模型'></a>
### Word2vec模型
[上一级](#语义特征)

In [25]:
### 训练词向量
def W2vec():
    global w2model
    model_path = 'word2vec.txt'
    if os.path.exists(model_path):
        w2model = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(model_path)
    else:
        w2model = Word2Vec(LineSentence('enwiki-latest-pages-articles.txt'), size=128, window=5, min_count=3, workers=multiprocessing.cpu_count())
        w2model.wv.save_word2vec_format(model_path)
    return w2model.wv

In [26]:
model = W2vec()

2018-11-15 17:53:15,427 - utils_any2vec.py[line:170] - INFO: loading projection weights from word2vec.txt
2018-11-15 17:53:15,430 - smart_open_lib.py[line:176] - DEBUG: {'kw': {}, 'mode': 'rb', 'uri': 'word2vec.txt'}
2018-11-15 17:59:09,538 - utils_any2vec.py[line:232] - INFO: loaded (3306942, 128) matrix from word2vec.txt
/home/lb/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


<a id='语义相似度'></a>
### 语义相似度
[上一级](#语义特征)

In [27]:
def Similarity(model, topword, keyword):
    if topword == []:
        return 0
    topword = [w for w in topword if w in model.vocab]
    return model.n_similarity(topword, keyword)

### 主题模型
[上一级](#主题特征)

In [28]:
# 训练、保存、读取 主题模型
def LDA(df):
    dictionary = corpora.Dictionary(df['topword'])
    corpus = [dictionary.doc2bow(text) for text in df['topword']]
    ldamodel = 'lda.model'
    if os.path.exists(ldamodel):
        lda = gensim.models.LdaModel.load(ldamodel)
    else:
        lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=6, passes=20)
        lda.save(ldamodel)
    corpus_lda = lda[corpus]
    lda_prop = list(corpus_lda)
    df['lda_prop'] = lda_prop
    return lda, df

<a id='主题相似度'></a>
### 主题相似度
[上一级](#主题特征)

In [29]:
# 主题相似度
def Combine(label, corpus_lda):
    for l in corpus_lda:
        if l[0] == label:
            return l[1]
    return 0

<a id='主题词'></a>
### 主题词个数
[上一级](#主题特征)

In [30]:
# 主题词出现次数
def Theme_count(ldaword, reserved_word):
    count = 0
    for word in reserved_word:
        for t in ldaword:
            if STEM.stem(word) == STEM.stem(t):
                count += 1
    return count/(len(reserved_word)+0.1)

<a id='读取数据'></a>
## 读取数据
[返回顶部](#顶部)

In [31]:
# 读取数据并删除多余空格
def Load_data():
    df = pd.read_excel('写作数据.xlsx')
    df['UserAnswer'] = df['UserAnswer'].map(lambda x: x.strip())
    return df

<a id='数据预处理'></a>
## 数据预处理
[1.粗分词。](#粗分词)
[2.细分词。](#细分词)
[3.分句。](#文章分句)
[4.去停用词。](#去停用词)
[5.筛选词性。](#筛选词性)
[6.保留top词(tfidf)。](#tfidf)<br>
[返回顶部](#顶部)

In [32]:
def Pretreatment(df):
    df['all_word'] = df['UserAnswer'].map(Wordtokenizer)
    df['word'] = df['UserAnswer'].map(Pure_word)
    df['sentence'] = df['UserAnswer'].map(SplitSentence)
    df['reserved_word'] = df['word'].map(Delete_stop)
    #IDF = Idf(df)
    #df['topword'] = df['reserved_word'].map(lambda x: Tfidf(IDF, x))
    df['topword'] = df['reserved_word'].map(Pos_select)
    return df

<a id='统计特征'></a>
## 统计特征
[1.句子个数。](#句子个数)[2.单词拼写错误个数。](#单词拼写)[3.错误标点个数。](#错误标点)[4.每句首字母是否大写。](#首字母大写)[5.文章是否写完。](#作文是否写完)[6.句子长度统计量：最大长度、最小长度、平均长度。](#句子长度统计)[7.字数限制。](#字数限制)[8.单词个数。](#1)[9.题目类型。](#1)[10.用词丰富度。](#用词丰富度)[11.词性统计](#词性统计)<br>
[返回顶部](#顶部)

In [33]:
def Statistical_Features(d1):
    d1['richness'] = d1['reserved_word'].map(Richness)
    d1['sentence_count'] = d1['sentence'].map(len)
    d1['wrong_count'] = d1['word'].map(Wrong_count)
    d1['wrong_signal'] = d1['UserAnswer'].map(Wrong_signal)
    d1['word_len'] = d1['word'].map(len)
    d1['label'] = d1['SysMeasureID'].map({'C4M01':0, 'C4M02':1, 'C4M03':2, 'C6M01':3, 'C6M02':4, 'C6M03':5})
    d1['limit_word'] = d1.apply(lambda item: Limit_count(item['SysMeasureID'], item['word_len']), axis=1)
    d1['initial_capitalization_count'] = d1['UserAnswer'].map(Initial_capitalization)
    d1['end'] = d1['UserAnswer'].map(End0)
    d1['sentence_len_max'] = d1['sentence'].map(lambda x: Statics(x, 'max'))
    d1['sentence_len_min'] = d1['sentence'].map(lambda x: Statics(x, 'min'))
    d1['sentence_len_mean'] = d1['sentence'].map(lambda x: Statics(x, 'mean'))
    #df['tag'] = df['reserved_word'].map(Tag_count)
    #tag = list(df['tag'])
    #tag = pd.DataFrame(tag)
    #tag.columns = ['tag'+str(i) for i in range(len(tag.columns))]
    #df = pd.concat([df, tag], axis=1)
    return d1

<a id='主题特征'></a>
## 主题特征
[1.主题模型。](#主题模型)[2.主题相似度。](#主题相似度)[3.主题词个数](#主题词个数)<br>
[返回顶部](#顶部)

In [34]:
def Theme_Features(df):
    lda, df = LDA(df)
    lda_word = lda.print_topics()
    keyword = ['university','ranking','certificate','study','Chinese','excessive','packaging','green','campus','online', 'shopping']
    ldaword = set(re.findall('[a-z]+', str(lda_word)) + keyword)
    df['lda'] = df.apply(lambda x: Combine(x['label'], x['lda_prop']), axis=1)
    df['themeword_count'] = df.apply(lambda x: Theme_count(ldaword, x['reserved_word']), axis=1)
    return df

<a id='语义特征'></a>
## 语义特征
[1.Word2vec模型。](#word2vec模型)[2.语义相似度。](#语义相似度)<br>
[返回顶部](#顶部)

In [35]:
def Semantics_Features(df):
    #model = W2vec()
    df['keyword'] = df['SysMeasureID'].map({'C4M01':['online','shopping'],'C4M02':['green','campus'],'C4M03':['excessive','packaging'],
                                            'C6M01':['study','chinese'],'C6M02':['university','ranking'],'C6M03':['certificate']})
    df['similarity'] = df.apply(lambda x: Similarity(model, x['topword'], x['keyword']), axis=1)
    return df

<a id='评价函数'></a>
### 评价函数F1
[返回顶部](#顶部)

In [36]:
def F1score(preds,dtrain):
    gaps = dtrain.get_label()
    f1 = metrics.f1_score(gaps, preds, average='weighted')
    return 'F1', f1

<a id='过采样'></a>
### 过采样
[返回顶部](#顶部)

In [37]:
def oversample(train, target):
    Oversampling0 = train.loc[train[target] == 0]
    Oversampling1 = train.loc[train[target] == 1]
    Oversampling2 = train.loc[train[target] == 2]
    Oversampling3 = train.loc[train[target] == 3]
    Oversampling4 = train.loc[train[target] == 4]
    Oversampling5 = train.loc[train[target] == 5]
    Oversampling6 = train.loc[train[target] == 6]
    Oversampling7 = train.loc[train[target] == 7]
    Oversampling8 = train.loc[train[target] == 8]
    m = len(Oversampling0)
    for i in range(round(m/len(Oversampling1))-1):
        train = train.append(Oversampling1)
    for j in range(round(m/len(Oversampling2))-1):
        train = train.append(Oversampling2)
    for k in range(round(m/len(Oversampling3))-1):
        train = train.append(Oversampling3)
    for k in range(round(m/len(Oversampling4))-1):
        train = train.append(Oversampling4)
    for k in range(round(m/len(Oversampling5))-1):
        train = train.append(Oversampling5)
    for k in range(round(m/len(Oversampling6))-1):
        train = train.append(Oversampling6)
    for k in range(round(m/len(Oversampling7))-1):
        train = train.append(Oversampling7)
    for k in range(round(m/len(Oversampling8))-1):
        train = train.append(Oversampling8)
    train.index = range(len(train))
    return train

<a id='XGBoost'></a>
### XGBoost模型，交叉验证
[返回顶部](#顶部)

In [38]:
def XGBoost(trainx, num, p, method): 
    target = 'Writing'
    train = trainx[trainx[target].notnull()]
    feature_name = [x for x in train.columns if x not in ['UserID','SysMeasureID','PlanID','UserAnswer','QText','Writing','OriginalText',
                                                           'all_word','word','sentence','reserved_word','lda_prop','label','topword','keyword']]   
    train, val, y, valy = train_test_split(train, train[target], test_size = 0.1,random_state=1) 
    #train = oversample(train, target)
    dtrain = xgb.DMatrix(train[feature_name], label = train[target], missing=np.nan)
    dval = xgb.DMatrix(val[feature_name], label = valy, missing=np.nan)
    watchlist  = [(dtrain,'train'),(dval,'val')]
    weight = Counter(y).most_common()
    weight = weight[0][1]/weight[-1][1]
    num_round = num
    params = {
                'booster':'gbtree',
                'objective': 'multi:softmax',
                'num_class': 9,
                'max_depth': 6,
                'gamma': 0.01,                
                'subsample': 0.85 ,       
                'colsample_bytree': 0.85,
                'scale_pos_weight': weight,
                'min_child_weight': 20,
                'eta': 0.1,
                'silent': 1,
            }   
    if p == 'train':
        #model = xgb.Booster(model_file='四六级.model')
        model = xgb.train(params, dtrain, num_round, evals=watchlist, verbose_eval=1, feval=eval(method))
        #model.save_model('四六级.model')
        result = pd.DataFrame({'truth': val[target], 'preds': model.predict(dval), 'UserAnswer': val['UserAnswer']})
        feature_score = model.get_fscore()
        feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
        fs = []
        for (key,value) in feature_score:
            fs.append("{0},{1}\n".format(key,value))
        with open('feature_score.csv','w') as f:
            f.writelines("feature,score\n")
            f.writelines(fs)
        return result, feature_score
    if p == 'cv':
        train = trainx[trainx[target].notnull()]
        dtrain = xgb.DMatrix(train[feature_name], label = train[target], missing=np.nan)
        model = xgb.cv(params, dtrain, num_round, nfold=10, feval = eval(method),callbacks=[xgb.callback.print_evaluation(show_stdv=True)])     
        return model['test-merror-mean']

In [39]:
def Main():
    df = Load_data()
    df = Pretreatment(df)
    df = Statistical_Features(df)
    df = Theme_Features(df)
    df = Semantics_Features(df)
    return df